In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
def fetch_page(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.text
    except requests.RequestException as e:
        print(f"An error occurred: {e}")
        return None

def extract_card_info(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    card_data = []
    
    cards = soup.find_all('div', {'class': 'w-80-ns'})   
    
    for card in cards:
        # Initialize dictionary to hold individual card's data
        card_info = {}
        
        
        card_name_section = card.find('h2')
        if card_name_section:
            card_info['Card Name'] = card_name_section.text.strip()
        
        # Extract rewards rate, annual fee, and welcome bonus
        # These are usually in a list, so we loop through them
        for li in card.find_all('li'):
            text = li.text.strip()
            if 'Rewards rate' in text:
                card_info['Rewards Rate'] = text.replace('Rewards rate', '')
            elif 'Annual fee' in text:
                card_info['Annual Fee'] = text.replace('Annual fee', '')
            elif 'Welcome bonus' in text:
                card_info['Welcome Bonus'] = text.replace('Welcome bonus', '')
            elif "Editors' bonus estimate" in text:
                card_info["Editors' bonus estimate"] = text.replace("Editors' bonus estimate", '')
            
        
        # Extract descriptions
        # descriptions_section = card.find('ul', {'class': 'descriptions'}) 
        # if descriptions_section:
        #     descriptions = [item.text.strip() for item in descriptions_section.find_all('li')]
        #     card_info['Descriptions'] = ', '.join(descriptions)
        
        # Append the card_info dictionary to card_data list
        if card_info:  
            card_data.append(card_info)
    
    # Convert the list of dictionaries to a Pandas DataFrame
    df = pd.DataFrame(card_data)
    
    return df


In [ ]:
# URL of the page to scrape
url = 'https://www.creditkarma.com/credit-cards/search-cc'

# Fetch the HTML content of the page
html_content = fetch_page(url)

if html_content:
    # Extract card info and convert it to a Pandas DataFrame
    df = extract_card_info(html_content)
    print(df)

In [ ]:
import json
def save_to_json(df, filename):
    df.to_json(filename, orient='records', lines=True)
    print(f"Data saved to {filename}")

In [ ]:
save_to_json(df, 'credit_card_basic_all.json')